# Materials Project Integration & VASP Input Generation

This notebook demonstrates how to fetch structures from the Materials Project
and generate complete VASP input files using CrystalMath.

You will learn how to:
1. Configure the Materials Project API key
2. Generate VASP inputs directly from MP IDs
3. Use different INCAR presets for various calculations
4. Generate custom k-point meshes
5. Work with pymatgen structures
6. Submit VASP jobs to CrystalMath

**Prerequisites:**
- `uv sync --all-extras` (includes mp-api and pymatgen)
- Materials Project API key (free from materialsproject.org)

## 1. Setup Materials Project API Key

You need a free API key from Materials Project.
Get one at: https://materialsproject.org/api

Configure it via environment variable or `~/.config/.pmgrc.yaml`

In [ ]:
import os

# Option 1: Set environment variable
# os.environ["MP_API_KEY"] = "your-key-here"

# Option 2: Configure in ~/.config/.pmgrc.yaml (recommended)
# PMG_MAPI_KEY: your-key-here

# Check if configured
api_key = os.environ.get("MP_API_KEY")
if api_key:
    print(f"API key configured: {api_key[:8]}...")
else:
    print("No API key in environment. Will try ~/.config/.pmgrc.yaml")

## 2. Generate VASP Inputs from Materials Project

The `generate_vasp_inputs_from_mp()` function fetches a structure
and generates complete VASP inputs in one step.

In [ ]:
from crystalmath.vasp.generator import (
    VaspInputGenerator,
    generate_vasp_inputs_from_mp,
)
from crystalmath.vasp.incar import IncarPreset

# Generate VASP inputs for Silicon (mp-149)
try:
    inputs = generate_vasp_inputs_from_mp(
        mp_id="mp-149",
        preset=IncarPreset.RELAX,
        encut=520.0,
        kppra=2000,
    )
    
    print("=== POSCAR ===")
    print(inputs.poscar[:300] + "...")
    
    print("\n=== INCAR ===")
    print(inputs.incar)
    
    print("\n=== KPOINTS ===")
    print(inputs.kpoints)
    
    print("\n=== POTCAR symbols ===")
    print(inputs.potcar_symbols)
    
except ImportError as e:
    print(f"Error: {e}")
    print("Install with: uv sync --all-extras")
except Exception as e:
    print(f"Error fetching from MP: {e}")
    print("Check your API key configuration")

## 3. INCAR Presets

CrystalMath provides preset configurations for common calculation types:
- `STATIC` - Single-point energy
- `RELAX` - Ionic relaxation
- `RELAX_CELL` - Cell + ion relaxation
- `BANDS` - Band structure (non-SCF)
- `DOS` - Density of states
- `DIELECTRIC` - Dielectric properties (DFPT)

In [ ]:
from crystalmath.vasp.incar import IncarBuilder, IncarPreset

# Show all available presets
print("Available INCAR presets:\n")

for preset in IncarPreset:
    builder = IncarBuilder.from_preset(preset)
    incar_str = builder.to_string()
    
    print(f"--- {preset.value.upper()} ---")
    print(incar_str[:200] + "...\n")

## 4. Custom K-points

Generate k-point meshes with different methods:
- Gamma-centered
- Monkhorst-Pack
- Automatic from density (kppra)

In [ ]:
from crystalmath.vasp.kpoints import KpointsBuilder, KpointsMesh

# Gamma-centered mesh (recommended for metals)
mesh_gamma = KpointsBuilder.gamma_centered(8, 8, 8)
print("=== Gamma-centered 8x8x8 ===")
print(mesh_gamma.to_string())

# Monkhorst-Pack mesh (general purpose)
mesh_mp = KpointsBuilder.monkhorst_pack(6, 6, 6)
print("\n=== Monkhorst-Pack 6x6x6 ===")
print(mesh_mp.to_string())

# Automatic from structure (requires pymatgen)
try:
    from pymatgen.core import Structure
    
    # Load a structure
    structure = Structure.from_file("POSCAR") if os.path.exists("POSCAR") else None
    
    if structure:
        mesh_auto = KpointsBuilder.from_density(structure, kppra=2000)
        print("\n=== Automatic (kppra=2000) ===")
        print(mesh_auto.to_string())
except ImportError:
    print("\nAutomatic mesh requires pymatgen (uv sync --all-extras)")

## 5. Generate from Local Structure

If you have a pymatgen Structure object or CIF file,
you can generate VASP inputs directly.

In [ ]:
try:
    from pymatgen.core import Structure, Lattice
    
    # Example: Create a simple Si structure programmatically
    lattice = Lattice.cubic(5.43)
    si = Structure(
        lattice,
        ["Si", "Si"],
        [[0, 0, 0], [0.25, 0.25, 0.25]]
    )
    
    # Generate VASP inputs
    gen = VaspInputGenerator(
        structure=si,
        preset=IncarPreset.STATIC,
        encut=400.0,
        kppra=1000,
    )
    
    inputs = gen.generate()
    
    print("Generated VASP inputs from pymatgen Structure")
    print(f"POSCAR: {len(inputs.poscar.splitlines())} lines")
    print(f"INCAR:  {len(inputs.incar.splitlines())} lines")
    print(f"KPOINTS: {len(inputs.kpoints.splitlines())} lines")
    print(f"Elements: {inputs.potcar_symbols}")
    
    # Show POSCAR
    print("\n=== Generated POSCAR ===")
    print(inputs.poscar)
    
except ImportError:
    print("pymatgen not installed")
    print("Install with: uv sync --all-extras")

## 6. Submit VASP Job to CrystalMath

Once you have generated inputs, submit them as a job.
The POSCAR goes in `input_content`, and auxiliary files
(INCAR, KPOINTS) go in the `auxiliary_files` dict.

In [ ]:
from crystalmath.api import CrystalController
from crystalmath.models import JobSubmission, DftCode, RunnerType

# Initialize controller
ctrl = CrystalController(db_path="vasp_demo.db", use_aiida=False)

# Submit with generated inputs
if 'inputs' in locals():
    job = JobSubmission(
        name="si_relax_mp149",
        dft_code=DftCode.VASP,
        runner_type=RunnerType.LOCAL,
        input_content=inputs.poscar,
        auxiliary_files={
            "INCAR": inputs.incar,
            "KPOINTS": inputs.kpoints,
        },
        parameters={"potcar_symbols": inputs.potcar_symbols},
    )
    
    pk = ctrl.submit_job(job)
    print(f"Submitted VASP job: PK={pk}")
    
    # Check status
    details = ctrl.get_job_details(pk)
    if details:
        print(f"Job state: {details.state.value}")
        print(f"Work dir: {details.work_dir}")
else:
    print("No inputs generated (skipping submission)")

## 7. Batch Generation for Multiple Materials

Generate VASP inputs for multiple materials in one go.

In [ ]:
# List of materials to process
materials = [
    {"mp_id": "mp-149", "name": "Si", "preset": IncarPreset.RELAX},
    {"mp_id": "mp-32", "name": "Ge", "preset": IncarPreset.RELAX},
    {"mp_id": "mp-2534", "name": "GaAs", "preset": IncarPreset.STATIC},
]

print("Generating VASP inputs for multiple materials:\n")

for mat in materials:
    try:
        inputs = generate_vasp_inputs_from_mp(
            mp_id=mat["mp_id"],
            preset=mat["preset"],
            encut=520.0,
            kppra=2000,
        )
        
        print(f"✓ {mat['name']:8s} ({mat['mp_id']}):")
        print(f"  Preset: {mat['preset'].value}")
        print(f"  Elements: {', '.join(inputs.potcar_symbols)}")
        print(f"  POSCAR lines: {len(inputs.poscar.splitlines())}")
        print()
        
    except Exception as e:
        print(f"✗ {mat['name']:8s} ({mat['mp_id']}): Error - {e}\n")

## Summary

In this notebook, you learned how to:

1. **Configure** the Materials Project API key
2. **Generate** complete VASP inputs from MP IDs
3. **Use** INCAR presets for different calculation types
4. **Create** custom k-point meshes (Gamma, MP, automatic)
5. **Work with** pymatgen structures
6. **Submit** VASP jobs to CrystalMath
7. **Batch process** multiple materials

**Key Classes:**
- `VaspInputGenerator` - Main input generator
- `IncarBuilder` - INCAR file builder with presets
- `KpointsBuilder` - K-point mesh generator
- `VaspInputs` - Dataclass for complete input set

**INCAR Presets:**
- `STATIC` - Single-point energy
- `RELAX` - Ionic relaxation (ISIF=2)
- `RELAX_CELL` - Cell + ion relaxation (ISIF=3)
- `BANDS` - Band structure (ICHARG=11)
- `DOS` - Density of states
- `DIELECTRIC` - Dielectric properties (DFPT)

**Next Steps:**
- `03_band_structure.ipynb` - Band structure workflows
- `04_convergence_eos.ipynb` - Convergence studies and equation of state
- `05_templates_workflows.ipynb` - Template system and phonon workflows